In [54]:
import pandas as pd
import sqlite3
import re
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from underthesea import word_tokenize

In [4]:
conn = sqlite3.connect("../data/vnexpress.db")
df = pd.read_sql_query("SELECT * FROM ARTICLE", conn)
len(df)

1048

In [21]:
category_counts = df["category"].value_counts()
valid_categories = category_counts[category_counts >= 2].index
df = df[df['category'].isin(valid_categories)]

In [22]:
X = df["content"]
y = df["category"]

In [11]:
stopwords = set([
    "tôi", "anh", "chị", "em", "bạn", "chúng", "ta", "họ", "mình", "bạn", "nó",
    "này", "kia", "đó", "ấy", "nọ", "này", "đấy", "đây", "đấy", "đó", "này",
    "là", "của", "và", "có", "nhưng", "thì", "lại", "nên", "vẫn", "được", "bị", "phải", "rằng", "nếu", "vì", "khi", "ở",
    "đã", "đang", "sẽ", "đến", "với", "từ", "do", "trong", "ngoài", "trên", "dưới", "theo", "qua", "để", "cũng", "đó", 
    "nữa", "còn", "hay", "hoặc", "mà", "nhé", "thôi", "luôn", "chỉ", "hơn", "kém", "đều",
    "một", "hai", "ba", "nhiều", "ít", "rất", "hơi", "vài", "mỗi", "các", "mọi", "tất", "cả",
    "và", "hoặc", "nhưng", "bởi", "do", "tuy", "dù", "mặc", "dù", "xong", "để", "cũng",
    "gì", "ai", "đâu", "sao", "nào", "khi nào", "tại sao",
    "vâng", "dạ", "ừ", "ờ", "ồ", "à", "á", "ồ", "vậy", "thế", "thật", "ừm", "ơ", "ha", "hả", "à", "nhỉ", "chắc",
    "cái", "việc", "này", "kì", "sự", "lúc", "nơi", "người", "điều", "chuyện", "trường", "thời", "gian", "năm", "tháng"
])

In [12]:
def preprocess_text(text):
    text = re.sub(r"[^\w\s]", " ", text.lower())
    tokens = word_tokenize(text, format="text").split()
    filtered = [word for word in tokens if word not in stopwords and len(word) > 1]
    return " ".join(filtered)

In [23]:
X = X.fillna("").apply(preprocess_text)
# Vecto hoá dữ liệu
vectorizer = TfidfVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(X)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y, test_size=0.2, random_state=19, stratify=y
)

In [26]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [27]:
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8714285714285714
                    precision    recall  f1-score   support

      Bất động sản       0.95      0.87      0.91        23
           Du lịch       0.95      0.88      0.91        24
          Giáo dục       0.95      0.95      0.95        22
          Giải trí       0.95      0.90      0.92        20
Khoa học công nghệ       1.00      0.27      0.43        11
        Kinh doanh       0.81      0.90      0.85        29
          Sức khỏe       0.96      0.88      0.92        26
          Thể thao       1.00      1.00      1.00        22
           Thời sự       0.64      0.88      0.74        33

          accuracy                           0.87       210
         macro avg       0.91      0.84      0.85       210
      weighted avg       0.89      0.87      0.87       210



In [47]:
def predict_category(text):
    processed = preprocess_text(text)
    vectorized = vectorizer.transform([processed])
    return model.predict(vectorized)[0]

### `Gradio`

In [ ]:
iface = gr.Interface(
    fn= predict_category,
    inputs= gr.Textbox(lines=5, placeholder="Nhập đoạn văn bạn cần phân loại thể loại"),
    outputs="text",
    title ="Dự đoán chuyên mục bài báo",
)
iface.launch()

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.42.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
